In [1]:
import os 
os.chdir(fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3')

print(os.getcwd())

\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3


In [96]:
# creates a list of paths (path_list) with working files

import os


# Create a list of paths the name of which has two variables: mNRA_Name and SubSET
# def get_path(mNRA_Red, mRNA_Green, SubSET):
def get_path():

    input_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'
    
    path_list = []

    # Loop through all directories and subdirectories inside the input directory
    for root, dirs, files in os.walk(input_dir):
        # Loop through all files in the current directory
        for file in files:
            # Check if the file has an _info.txt extension and starts with the folder name
            if "_info.txt" in file and file.startswith(os.path.basename(root)):
                # Read the file content and convert it to lowercase
                
                path_list.append(root)

    # Return the list of paths
    return path_list


In [97]:
path_list = get_path()

In [98]:
# runs function get_path and saves path list as .txt file

path_list = get_path()

with open('path_list.txt', 'w') as file:
    for item in path_list:
        file.write(item + '\n')
        
len(path_list)

602

In [5]:
path_list

['\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total\\2023-09-26_C57BL6_M_Control-4h_Camk2d-488_NCad-568_Calm1-647_c1_deconv',
 '\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total\\2023-09-26_C57BL6_M_Control-4h_Camk2d-488_NCad-568_Calm1-647_c2_deconv',
 '\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total\\2023-09-26_C57BL6_M_Control-4h_Camk2d-488_NCad-568_Calm1-647_c3_deconv',
 '\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total\\2023-09-26_C57BL6_M_Control-4h_Camk2d-488_NCad-568_Calm1-647_c4_deconv',
 '\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total\\2023-09-26_C57BL6_M_Control-4h_Camk2d-488_NCad-568_Calm1-647_c5_deconv',
 '\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total\\2023-09-26_C57BL6_M_Control-4h_Camk2d-488_NCad-568_Calm1-647_c6_deconv',
 '\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total\\2023-09-26_C57BL6_M_Control-4h_Camk2d-488_NCad-568_Calm1-647_c7_deconv',
 '\\\\?\\D:\\Projects\\Calm_ISO\\5h_Injection\\ISO_Total\\2023-09-26_C57BL6_M_Contr

In [6]:
# Creates 3D images with light background

import os
import napari
import numpy as np
import tifffile
import pathlib
import re
from skimage import io, util
from scipy import ndimage
import pyautogui
import time
from tifffile import TiffFileError

os.chdir(fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3')

def Inverted_MAX_Projection(path):

    for path in path_list:

        with open(os.path.join(path, fr'' + os.path.basename(path) + '_info.txt'), 'r', encoding='utf-8', errors='ignore') as file:
            content = file.read()

        pattern = r"XYZCalibration\(um\/vox\):(\d+\.\d+),(\d+\.\d+),(\d+\.\d+):"
        match = re.search(pattern, content)
 
        if match:
            x_dim = float(match.group(1))
            y_dim = float(match.group(2))
            z_dim = float(match.group(3))

        # Extract channel numbers and names using regular expressions
#         channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+)', content)[0]))
#         channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+)', content)[0]
#         experimental_group = re.findall(r'ExperimentalGroup:(\w+)', content)[0]
        
        channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+),(\d+)', content)[0])) #ONLY for 3rd channel
        channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+),(\w+)', content)[0] #ONLY for 3rd channel      
        experimental_group = re.findall(r'ExperimentalGroup:(\w+)', content)[0]

        # Combine channel numbers and names into a dictionary
        channel_info = dict(zip(channel_numbers, channel_names))

        
        mask0 = channel_names[0]
        raw0 = channel_numbers[0]
        mask1 = channel_names[1]
        raw1 = channel_numbers[1]
        mask2 = channel_names[2]
        raw2 = channel_numbers[2]
        mask3 = channel_names[3]
        raw3 = channel_numbers[3]
        
#         color1 = "I Forest"
        color1 = "I Purple"
        color2 = "gray_r"
        color3 = "I Bordeaux"
            
        target1_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask1}_filtered_image.tif')
        target2_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask2}_filtered_image.tif')
        target3_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask3}_filtered_image.tif')

        body_mask_path = os.path.join(path, 'Segmented', 'Masked_Body_image.tif')
        nuclei_mask_path = os.path.join(path, 'Segmented', 'Masked_Nuclei_image.tif')
        
        try:
            target_mask1 = io.imread(target1_mask_path)
            target_mask2 = io.imread(target2_mask_path)
            target_mask3 = io.imread(target3_mask_path)
        except:
            print(f"'{target1_mask_path}' is not a valid TIFF file.")
            continue  # This will skip the rest of the current loop and proceed to the next file

        body_mask = io.imread(body_mask_path)
        nuclei_mask = io.imread(nuclei_mask_path)

        # overlap12 = target_mask1*target_mask2
        # overlap23 = target_mask2*target_mask3
        # overlap13 = target_mask3*target_mask1
        overlap123 = target_mask3*target_mask2*target_mask1

        #this code first removes and then adds layers of the top and bottom of the body mask TIFF
        body_shape = body_mask.shape[0]
        body_middle =  body_mask[1:body_shape-1]
        body_BOTTOM_contour = util.img_as_bool(np.zeros((1, body_middle.shape[1], body_middle.shape[2])))
        body_BOTTOM = util.img_as_uint(ndimage.binary_fill_holes(body_BOTTOM_contour))
        body_TOP_contour = util.img_as_bool(np.zeros((1, body_middle.shape[1], body_middle.shape[2])))
        body_TOP = util.img_as_uint(ndimage.binary_fill_holes(body_TOP_contour))

        body = np.vstack((body_BOTTOM,body_middle,body_TOP)) 

        white_layer = util.img_as_ubyte(np.zeros((body_shape, body_middle.shape[1], body_middle.shape[2])))
        background = util.img_as_uint(np.invert(white_layer))

        viewer = napari.view_image(background, gamma=2, iso_threshold = 0, rendering='iso', depiction='volume', blending='additive', colormap='gray', name='background', scale=(z_dim, y_dim, x_dim))

        viewer.add_image(target_mask1, gamma=3, rendering='attenuated_mip',depiction='volume', blending='minimum', colormap= color1, name=f'{mask1}', scale=(z_dim, y_dim, x_dim))
        viewer.add_image(target_mask2, gamma=3, rendering='attenuated_mip',depiction='volume', blending='minimum', colormap= color2, name=f'{mask2}', scale=(z_dim, y_dim, x_dim))
        viewer.add_image(target_mask3, gamma=2, rendering='attenuated_mip',depiction='volume', blending='minimum', colormap= color3, name=f'{mask3}', scale=(z_dim, y_dim, x_dim))

        viewer.add_image(overlap123, gamma=2, interpolation3d  = 'spline36', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent', colormap='cyan', name='overlap', scale=(z_dim, y_dim, x_dim))
       
        # viewer.add_image(overlap12, gamma=2, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent_no_depth', colormap='gray', name='overlap', scale=(z_dim, y_dim, x_dim))
        # viewer.add_image(overlap13, gamma=2, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent_no_depth', colormap='gray', name='overlap', scale=(z_dim, y_dim, x_dim))#     viewer.add_image(overlap12, gamma=0.1, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent', colormap='yellow', name=f'{mask1}x{mask2}', scale=(z_dim, y_dim, x_dim))
        # viewer.add_image(overlap23, gamma=2, interpolation3d  = 'linear', rendering='attenuated_mip',depiction='volume', contrast_limits=[0, 1], blending='translucent_no_depth', colormap='gray', name='overlap', scale=(z_dim, y_dim, x_dim))
            
        viewer.add_image(nuclei_mask, gamma=0.08, interpolation3d  = 'nearest', iso_threshold = 0, rendering='iso', depiction='volume', opacity=0.9, blending='translucent', colormap='gray_r', name='nuclei', scale=(z_dim, y_dim, x_dim))
        viewer.add_image(body, gamma=0.7, interpolation3d  = 'kaiser', iso_threshold = 0, rendering='iso', depiction='volume', blending='minimum', opacity=0.15, colormap='gray', name='body', scale=(z_dim, y_dim, x_dim))

    #     Create a scale bar with a specified size and color
        viewer.scale_bar.visible = True
        viewer.scale_bar.color = "black"
#         viewer.scale_bar.font_size = 47
    #     viewer.scale_bar.colored = True
        viewer.scale_bar.unit = "um"
        viewer.theme = 'light' #for "dark"
        viewer.dims.ndisplay = 3
        viewer.camera.zoom = 20
        viewer.text_overlay.text = f'{experimental_group}\n {mask1}: {color1}\n {mask2}: {color2}\n {mask3}: {color3}\n Overlap: Cyan''
        viewer.text_overlay.color = 'black' # or  (0.5, 0.5, 0.5, 1.0)
        viewer.text_overlay.font_size = 27
        viewer.text_overlay.position = "top_center" # "top_left", "top_right", "top_center", "bottom_right", "bottom_left"
        viewer.text_overlay.visible = True
        viewer.window.resize(3840, 2160)
    
        from napari.utils.notebook_display import nbscreenshot

        nbscreenshot(viewer)
        im = viewer.screenshot(size =(2160, 3840))
        io.imsave(path + fr'_{mask1}_{mask2}_{mask3}_{experimental_group}.png', im)

        viewer.close()

    return

In [7]:
Inverted_MAX_Projection(path_list)